# Preprocess Data (Milestone 2)

---

In this notebook, we will edit the data to include only columns that we want. 

There are two types of variables that we want to keep:
- Those which are practical for the user to input when looking up a price prediction
- Those which would actually contribute to the price of a flight

---

## Data Dictionary Reminder

below is a reminder of what variables we have, next to each we indicate what we want to do with each column

- legId: An identifier for the flight. // Not needed
- searchDate: The date (YYYY-MM-DD) on which this entry was taken from Expedia. // Change search date and flight date into one variable - days before flight
- flightDate: The date (YYYY-MM-DD) of the flight. // Potentially separate this into just the day. Month is meaningless as the data is not even for a full year
- startingAirport: Three-character IATA airport code for the initial location. // One hot encoding
- destinationAirport: Three-character IATA airport code for the arrival location. // One hot encoding
- fareBasisCode: The fare basis code. // Not required
- travelDuration: The travel duration in hours and minutes. // Not required
- elapsedDays: The number of elapsed days (usually 0). // Not required
- isBasicEconomy: Boolean for whether the ticket is for basic economy. // Change into binary
- isRefundable: Boolean for whether the ticket is refundable. // Change into binary 
- isNonStop: Boolean for whether the flight is non-stop. // Change into binary 
- baseFare: The price of the ticket (in USD). // Not required
- totalFare: The price of the ticket (in USD) including taxes and other fees. // No change needed
- seatsRemaining: Integer for the number of seats remaining. // Not required
- totalTravelDistance: The total travel distance in miles. This data is sometimes missing. // Not required
- segmentsDepartureTimeEpochSeconds: String containing the departure time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsDepartureTimeRaw: String containing the departure time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsArrivalTimeEpochSeconds: String containing the arrival time (Unix time) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsArrivalTimeRaw: String containing the arrival time (ISO 8601 format: YYYY-MM-DDThh:mm:ss.000±[hh]:00) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsArrivalAirportCode: String containing the IATA airport code for the arrival location for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsDepartureAirportCode: String containing the IATA airport code for the departure location for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsAirlineName: String containing the name of the airline that services each leg of the trip. The entries for each of the legs are separated by '||'. // Most trips use the same airlines the whole way, drop trips that don't do this and change trips that do into one hot encoding. We can also use this to count how many layovers there are.
- segmentsAirlineCode: String containing the two-letter airline code that services each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsEquipmentDescription: String containing the type of airplane used for each leg of the trip (e.g. "Airbus A321" or "Boeing 737-800"). The entries for each of the legs are separated by '||'. // Not required
- segmentsDurationInSeconds: String containing the duration of the flight (in seconds) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsDistance: String containing the distance traveled (in miles) for each leg of the trip. The entries for each of the legs are separated by '||'. // Not required
- segmentsCabinCode: String containing the cabin for each leg of the trip (e.g. "coach"). The entries for each of the legs are separated by '||'. // Not required

## Load Spark and Data

In [182]:
from itertools import groupby
from math import radians, cos, sin, sqrt, atan2

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, dayofmonth, when, udf, split, size
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

from pyspark.sql.types import IntegerType, StringType, FloatType

In [183]:
spark = SparkSession.builder.appName("flights").getOrCreate()

# REPLACE WITH DATA FILEPATH
DATA_PATH = "../data/itineraries.csv"

raw_df = spark.read.csv(DATA_PATH, header=True, inferSchema=True)

## Drop the Uneccessary Columns

In [184]:
# added back in 
# "segmentsArrivalTimeRaw", "segmentsDepartureTimeRaw"

In [185]:
cols_to_drop = ["legId", "fareBasisCode", "travelDuration", "elapsedDays", "baseFare", "seatsRemaining", "totalTravelDistance",\
                "segmentsDepartureTimeEpochSeconds", "segmentsArrivalTimeEpochSeconds", \
                "segmentsArrivalAirportCode", "segmentsDepartureAirportCode","segmentsAirlineCode", "segmentsEquipmentDescription", \
                "segmentsDurationInSeconds", "segmentsDistance", "segmentsCabinCode"]

In [186]:
# drop cols
df = raw_df.drop(*cols_to_drop)

In [187]:
df.show()

+----------+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+
|searchDate|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw| segmentsAirlineName|
+----------+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+
|2022-04-16|2022-04-17|            ATL|               BOS|         false|       false|     true|    248.6|    2022-04-17T12:57:...|  2022-04-17T15:26:...|               Delta|
|2022-04-16|2022-04-17|            ATL|               BOS|         false|       false|     true|    248.6|    2022-04-17T06:30:...|  2022-04-17T09:00:...|               Delta|
|2022-04-16|2022-04-17|            ATL|               BOS|         false|       false|     true|    248.6|    2022-04-17

## Days before flight Column

In [188]:
df = df.withColumn("days_before_flight", datediff(col("flightDate"), col("searchDate")))

## DayofMonth Column

In [189]:
df = df.withColumn("day", dayofmonth(col("flightDate")))

## Drop: searchDay and FlightDay

In [190]:
#df = df.drop(*["searchDate", "flightDate"])
df = df.drop("searchDate")

In [191]:
df.show()

+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+------------------+---+
|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw| segmentsAirlineName|days_before_flight|day|
+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+------------------+---+
|2022-04-17|            ATL|               BOS|         false|       false|     true|    248.6|    2022-04-17T12:57:...|  2022-04-17T15:26:...|               Delta|                 1| 17|
|2022-04-17|            ATL|               BOS|         false|       false|     true|    248.6|    2022-04-17T06:30:...|  2022-04-17T09:00:...|               Delta|                 1| 17|
|2022-04-17|            ATL|               BOS|         fals

## One Hot Encode Starting and Destination Airport

In [192]:
# string indexers
string_indexer = StringIndexer(inputCol="startingAirport", outputCol="startingAirport_index")
string_indexer_2 = StringIndexer(inputCol="destinationAirport", outputCol="destinationAirport_index")

# encoders
encoder = OneHotEncoder(inputCol="startingAirport_index", outputCol="startingAirport_encoded")
encoder_2 = OneHotEncoder(inputCol="destinationAirport_index", outputCol="destinationAirport_encoded")

# pipeline
pipeline = Pipeline(stages=[string_indexer, string_indexer_2, encoder, encoder_2])
pipeline_model = pipeline.fit(df)
df = pipeline_model.transform(df)


## Drop: startingAirport, destinationAirport, startingAirport_index, destinationAirport_index

In [193]:
"""
df = df.drop(*["startingAirport", "destinationAirport", "startingAirport_index", "destinationAirport_index"])
"""
df = df.drop(*["startingAirport_index", "destinationAirport_index"])

## Convert Boolean Columns into Binary

In [194]:
bool_columns = ["isBasicEconomy", "isRefundable", "isNonStop"]

In [195]:
for col_name in bool_columns:
    df = df.withColumn(col_name, when(col(col_name), 1).otherwise(0))

In [196]:
df.show()

+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+------------------+---+-----------------------+--------------------------+
|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw| segmentsAirlineName|days_before_flight|day|startingAirport_encoded|destinationAirport_encoded|
+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+--------------------+------------------+---+-----------------------+--------------------------+
|2022-04-17|            ATL|               BOS|             0|           0|        1|    248.6|    2022-04-17T12:57:...|  2022-04-17T15:26:...|               Delta|                 1| 17|         (15,[1],[1.0])|            (15,[2],[1.0])|
|2022-04-17|            ATL|               B

## Create Number of Flights and Airline Name
## TODO: FIX THIS

In [197]:
# THESE UDF FUNCTIONS DON'T SEEM TO WORK, IGNORE FOR NOW

def all_equal(iterable):
    g = groupby(iterable)
    return next(g, True) and not next(g, False)

# get airline name
def get_airline(segmentsAirlineName):
    splits = segmentsAirlineName.split("||")
    if all_equal(splits):
        return splits[0]
    else:
        return "Multi-Airline"

# get number of splits
def get_nsplits(segmentsAirlineName):
    splits = segmentsAirlineName.split("||")
    return len(splits)

# register as udf 
airline_udf = udf(get_airline)
nsplit_udf = udf(get_nsplits, IntegerType())

In [198]:
# convert segments airline name into a list
df = df.withColumn("segmentsAirlineName", split(col("segmentsAirlineName"), r'\|\|'))

# count the number of legs in the trip
df = df.withColumn("num_legs", size(col("segmentsAirlineName")))

# check whether all the flights in the list are the same
df = df.withColumn("All_Same", col("segmentsAirlineName")[0] == col("segmentsAirlineName")[1])

# change the null results to true (for flights with only one leg)
df = df.withColumn("All_Same", when(col("All_Same").isNull(), True).otherwise(col("All_Same")))

# get the name of the airline
df = df.withColumn("airline_name", col("segmentsAirlineName").getItem(0))

# filter out unneccesary columns
df = df.filter(col("All_Same") != False)

In [199]:
# drop segmentsAirlineName
df = df.drop("segmentsAirlineName")

In [200]:
# convert all_same into bool
df = df.withColumn("All_Same", when(col("All_Same"), 1).otherwise(0))

In [201]:
# convert airline_name into one hot encoded
string_indexer_3 = StringIndexer(inputCol="airline_name", outputCol="airline_name_index")
encoder_3 = OneHotEncoder(inputCol="airline_name_index", outputCol="airline_name_encoded")
pipeline_2 = Pipeline(stages=[string_indexer_3, encoder_3])

pipeline_model_2 = pipeline_2.fit(df)
df = pipeline_model_2.transform(df)

In [202]:
# drop airline_name, airline_name_index
df = df.drop(*["airline_name", "airline_name_index"])

In [203]:
df.show()

+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+
|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw|days_before_flight|day|startingAirport_encoded|destinationAirport_encoded|num_legs|All_Same|airline_name_encoded|
+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+
|2022-04-17|            ATL|               BOS|             0|           0|        1|    248.6|    2022-04-17T12:57:...|  2022-04-17T15:26:...|                 1| 17|         (15,[1],[1.0])|            (15,[2],[1.0])|       1|   

# Feature Engineering (Milestone 3)

## Distances instead of or in addition to One Hot Encoding Airport codes

Previously, we used one hot encoding to encode the beginning and ending airports. 

This is not ideal because not only does it require the model to learn the relationships between many different flight paths, but it does not represent one very important factor that impacts the flight of prices - distance

In [205]:
# Approximate coordinates of the airports (latitude, longitude)
airport_coords = {
    'ATL': (33.6367, -84.4281),
    'DFW': (32.8968, -97.0380),
    'DEN': (39.8561, -104.6737),
    'ORD': (41.9742, -87.9073),
    'LAX': (33.9416, -118.4085),
    'CLT': (35.2140, -80.9431),
    'MIA': (25.7959, -80.2870),
    'JFK': (40.6413, -73.7781),
    'EWR': (40.6895, -74.1745),
    'SFO': (37.6188, -122.375),
    'DTW': (42.2124, -83.3534),
    'BOS': (42.3656, -71.0096),
    'PHL': (39.8719, -75.2411),
    'LGA': (40.7769, -73.8740),
    'IAD': (38.9531, -77.4565),
    'OAK': (37.7213, -122.221),
}

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    R = 6371 # Radius of Earth in kilometers
    distance = R * c
    return distance

# Calculate the distance for each pair and store it in a dictionary
distances = {}
for airport1 in airport_coords:
    for airport2 in airport_coords:
        if airport1 != airport2:
            coord1 = airport_coords[airport1]
            coord2 = airport_coords[airport2]
            distance = haversine(coord1[0], coord1[1], coord2[0], coord2[1])
            key1 = f"{airport1}-{airport2}"
            key2 = f"{airport2}-{airport1}"
            distances[key1] = distance
            distances[key2] = distance

# Save distances between start and ending locations in a dictionary
dist_dict = {key: distances[key] for key in list(distances)}

In [206]:
from pyspark.sql.functions import concat_ws

In [207]:
df = df.withColumn("route", concat_ws("-",col("startingAirport"),col("destinationAirport")))

In [208]:
def get_dist(val):
    # Return the mapped value if it exists, else return the original value
    return dist_dict.get(val, val)

# instantiate the udf
get_dist_udf = udf(get_dist, FloatType())

In [209]:
df = df.withColumn("distance", get_dist_udf("route"))
df.show()

+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+-------+--------+
|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw|days_before_flight|day|startingAirport_encoded|destinationAirport_encoded|num_legs|All_Same|airline_name_encoded|  route|distance|
+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+-------+--------+
|2022-04-17|            ATL|               BOS|             0|           0|        1|    248.6|    2022-04-17T12:57:...|  2022-04-17T15:26:...|                 1| 17|         (15

In [210]:
# TODO: drop the route column
#df = df.drop(*["startingAirport", "destinationAirport", "route"])
df = df.drop("route")

## Day of Week, Time of Day

We add variables to include the day of the week and the time of the day into our model. These could prove essential because of patterns such as flights being more expensive towards the end of the week (as people leave for weekend trips) and towards later in the day (after people get off work). 

In [211]:
from pyspark.sql.functions import date_format, to_date

In [212]:
# retrieve the data of departure, get the weekday and convert it into numerical using stringindexer
df = df.withColumn("departure_date", col("segmentsDepartureTimeRaw").substr(1, 10))

# retrieve the stringed time of departure and convert it into an integer
df = df.withColumn("departure_time", col("segmentsDepartureTimeRaw").substr(12, 2))
df = df.withColumn("departure_hour", col("departure_time").cast("integer"))

In [213]:
df = df.withColumn("departure_dow", date_format("departure_date", "E"))

In [214]:
string_indexer_dow = StringIndexer(inputCol="departure_dow", outputCol="departure_dow_idx")
modelx = string_indexer_dow.fit(df)
df = modelx.transform(df)

In [215]:
# drop unneeded columns
unneeded = ["departure_date", "departure_time", "departure_dow", "flightDate", "segmentsDepartureTimeRaw", "segmentsArrivalTimeRaw"]
df = df.drop(*unneeded)

In [216]:
df.show()

+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+--------+--------------+-----------------+
|flightDate|startingAirport|destinationAirport|isBasicEconomy|isRefundable|isNonStop|totalFare|segmentsDepartureTimeRaw|segmentsArrivalTimeRaw|days_before_flight|day|startingAirport_encoded|destinationAirport_encoded|num_legs|All_Same|airline_name_encoded|distance|departure_hour|departure_dow_idx|
+----------+---------------+------------------+--------------+------------+---------+---------+------------------------+----------------------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+--------+--------------+-----------------+
|2022-04-17|            ATL|               BOS|             0|           0|        1|    248.6|    2022

## Population: A Proxy for Demand

The price of a ticket is heavily impacted by the total air traffic passing through a city. For example, a small airport may charge a lot more for a comparable flight because there are very few flights departing. We attempt to proxy this factor using the population of the departure and arrival cities. 

In [217]:
# dictionary encompassing the metropolitan airports surrounding each city

pop_dict = {
    'ATL': 6930423,
    'DFW': 7846293,
    'DEN': 3623560,
    'ORD': 9458539,
    'LAX': 13200998,
    'CLT': 2701046,
    'MIA': 6138333,
    'JFK': 19768458, # Representing the NY metropolitan area
    'LGA': 19768458, # Representing the NY metropolitan area
    'EWR': 19768458, # Representing the NY metropolitan area
    'SFO': 4749008,
    'DTW': 4392041,
    'BOS': 4941632,
    'PHL': 6245051,
    'IAD': 6385162, # Representing the Washington D.C. metropolitan area
    'OAK': 4749008  # Included in the San Francisco metropolitan area
}

# apply minmax scaling to get more digestable features 
pop_values = list(pop_dict.values())

# Calculating min and max for the scaling formula
min_pop = min(pop_values)
max_pop = max(pop_values)

scaled_pop_dict = {airport: ((population - min_pop) / (max_pop - min_pop)) * 100 
                      for airport, population in pop_dict.items()}

In [218]:
scaled_pop_dict

{'ATL': 24.780423651810832,
 'DFW': 30.14661508141949,
 'DEN': 5.405119417050459,
 'ORD': 39.59295644823011,
 'LAX': 61.520469535744496,
 'CLT': 0.0,
 'MIA': 20.139473987034474,
 'JFK': 100.0,
 'LGA': 100.0,
 'EWR': 100.0,
 'SFO': 11.999253313859185,
 'DTW': 9.907741138492469,
 'BOS': 13.127860275477033,
 'PHL': 20.764747461419457,
 'IAD': 21.58567450062142,
 'OAK': 11.999253313859185}

In [219]:
# apply this population dict to the 

def get_pop(val):
    # Return the mapped value if it exists, else return the original value
    return scaled_pop_dict.get(val, val)

# instantiate the udf
get_pop_udf = udf(get_pop, FloatType())

In [220]:
df = df.withColumn("starting_pop", get_pop_udf("startingAirport"))
df = df.withColumn("destination_pop", get_pop_udf("destinationAirport"))

In [221]:
# drop starting and destination airports
df = df.drop(*["startingAirport", "destinationAirport"])

In [226]:
df.show()

+--------------+------------+---------+---------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+--------+--------------+-----------------+------------+---------------+
|isBasicEconomy|isRefundable|isNonStop|totalFare|days_before_flight|day|startingAirport_encoded|destinationAirport_encoded|num_legs|All_Same|airline_name_encoded|distance|departure_hour|departure_dow_idx|starting_pop|destination_pop|
+--------------+------------+---------+---------+------------------+---+-----------------------+--------------------------+--------+--------+--------------------+--------+--------------+-----------------+------------+---------------+
|             0|           0|        1|    248.6|                 1| 17|         (15,[1],[1.0])|            (15,[2],[1.0])|       1|       1|      (12,[1],[1.0])|1521.342|            12|              0.0|   24.780424|       13.12786|
|             0|           0|        1|    248.6|               

## Save the final Data

In [204]:
# change to code cell to save

In [225]:
# Where the file should be saved
#TARGET_PATH = "../data/itineraries_processed.csv"
TARGET_PATH_PAR = "../data/itineraries_processed.parquet"

# save to csv
# df.toPandas().to_csv(TARGET_PATH, header=True, index=False)

# writing to parquet makes loading the data easier
df.write.parquet(TARGET_PATH_PAR)

In [227]:
spark.stop()